# Clustering Public Housing Estates in Hong Kong
### Capstone Project - The Battle of the Neighborhoods


## 1. Assignment's description

A description of the problem and a discussion of the background

A description of the data and how it will be used to solve the problem.A description of the data and how it will be used to solve the problem.



New Delhi is the capital city of India. It is a part of the city of Delhi’s 11 districts. The city itself has a population of 257,803. However, the much larger metro area has a population that exceeds 26 million.
New Delhi are used interchangeably to refer to the National Capital Territory of Delhi (NCT), these are two distinct entities, with New Delhi forming a small part of Delhi. The National Capital Region is a much larger entity comprising the entire NCT along with adjoining districts in neighboring states.
The official language of New Delhi and the one that is most widely spoken is Hindi. However, English is also spoken as a formal language within businesses and government agencies. Over last decades it is continuously grow because of the city’s important role in government and commercial business.
With it’s diverse culture , comes diverse food items. There are many restaurants in New Delhi City, each belonging to different categories like Chinese , Italian , French etc.
So as part of this project , we will list and visualize all major parts of New Delhi City .

Questions that can be asked using the above mentioned datasets
What is best location in New Delhi City for Chinese Cuisine ?
Which areas have large number of Chinese Resturant Market ?
Which all areas have less number of resturant ?
Which is the best place to stay if I prefer Chinese Cuisine ?
What places are have best restaurant in New Delhi?


For this project we need the following data :
New Delhi Resturants data that contains list Locality, Resturant name,Rating along with their latitude and longitude.
Data source : Zomato kaggel dataset : "https://www.kaggle.com/shrutimehta/zomato-restaurants-data"
Description : This data set contains the required information. And we will use this data set to explore various locality of new delhi city.
Nearby places in each locality of new delhi city.
Data source : Fousquare API : "https://developer.foursquare.com/"
Description : By using this api we will get all the venues in each neighborhood.



Blue Ocean Strategy - Positioned to general consumer, we aim to open a valued Fast Food Restaurant to the neighborhoods in one of the remote HK districts in Tuen Mun, Hong Kong.
We need to identify the exsiting gap if there are demand but no or not many restaurants.

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

According to the HK Census and Statistics Department, overall HK restaurant earnings recorded strong decline of **29.4%** growth to HK$79.4 billion due to the Covid-19 pandemic and sweeping social-distancing regulations.

Despite this, Hong Kongers cannot resist eating out and we would like to tap into this market and get ready the **post Covid-19**.

In order to launch a successful business we need to plan carefully beforehand, which is the **location, location and location**.

With **Blue Ocean Strategy** in mind, we will identify the existing gap to see if there are competition in the neighborhoods.

Positioned to general consumer, we aim to open a **valued Fast Food Restaurant** to the neighborhoods where are located in the remote HK district **Tuen Mun**, Hong Kong. Hong Kong is divided into 431 District Council Constituency Areas under 18 Districts.

## Data <a name="data"></a>

* Data source: HK Open data "https://data.gov.hk/en/" it is used to identify demand based on people lives in the estates.

* Data source : Fousquare API : "https://developer.foursquare.com/" It is used to fetch all the venues in each neighborhood so that we could identify if there are major competition.

### Scrap content from Hong Kong Open Data
Import necessary packages.

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import re

Get the latitude and the longitude coordinates of each HK Housing Estate from HK Open Data

In [16]:
r = requests.get('https://data.housingauthority.gov.hk/psi/rest/export/prh-estates')
x = r.json()

x.keys()
df = pd.DataFrame(x['data'])
df

seelct field Estate_Name, District_Name, Map_Latitude, Map_Longitude



,Estate_Name,District_Name,Region_Name,Map_Latitude,Map_Longitude,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,Property_Management,Carpark_Management,Estate_Website,Further_Information
0,Tak Long Estate,Kowloon City,Kowloon West,22.330105,114.2031,Public Rental Housing,2013,Non-standard,9,Tak Cheung House Tak Sui House Tak Kei House T...,8 200 As at 31.3.2021,14 – 37,Formed,Kowloon West & Sai Kung District Tenancy Manag...,Good Excel Property Consultant Ltd G/F Carpor...,Good Excel Property Consultant Ltd G/F Carpor...,,<a href= /en/common/pdf/global-elements/estate...
1,Kai Ching Estate,Kowloon City,Kowloon West,22.33254,114.20292,Public Rental Housing,2013,Non-standard,6,Hong Ching House Lok Ching House Yan Ching Hou...,5 200 As at 31.3.2021,14.05-37.58,Formed,Kowloon West & Sai Kung District Tenancy Manag...,Good Excel Property Consultants Ltd. 1/F Hong...,Good Excel Property Consultants Ltd. 1/F Hong...,-,<a href= /en/common/pdf/global-elements/estate...
2,Choi Fook Estate,Kwun Tong,Kowloon East,22.328026,114.21717,Public Rental Housing,2010 2021,Non-standard,5,Choi Lok House Choi Sin House Choi Hay House C...,4 500 As at 31.3.2021,21.2 - 43.6,Formed,Housing Department Kowloon East (12) District ...,Good Excel Property Consultants Ltd. G/F Cho...,Wilson Parking (Holdings) Limited Room 2901 2...,,<a href= /en/common/pdf/global-elements/estate...
3,Ching Ho Estate,North,New Territories East,22.49311,114.12577,Public Rental Housing,2006,New Harmony ( I ),8,Ching Chak House Ching Chiu House Ching Chung ...,7 200 As at 31.3.2021,17.9 - 39.8,Formed,Tai Po North & Shatin District Tenancy Manage...,Creative Property Services Consultants Limited...,Yue Xiu APT Parking Ltd 28/F Seabright Plaza ...,--,<a href= /en/common/pdf/global-elements/estate...
4,Hin Yiu Estate,Sha Tin,New Territories East,22.36692,114.17457,Public Rental Housing,2005,New Harmony I,1,Hin Yiu House,800 As at 31.3.2021,17.81 - 39.74,Not Yet Formed,Tai Po North & Shatin District Tenancy Manage...,Creative Property Services Consultants Limited...,Creative Property Services Consultants Limited...,-,<a href= /en/common/pdf/global-elements/estate...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Shek Yam East Estate,Kwai Tsing,New Territories West,22.3727,114.13965,Public Rental Housing,1996,Harmony 1,3,Yam Heng House Yam Hing House Yam Yue House,2 400 As at 31.3.2021,13.6 - 49.0,Formed,Shek Yam East Estate Office G/F Yam Hing Hous...,Shek Yam East Estate Office G/F Yam Hing Hous...,,-,<a href= /en/common/pdf/global-elements/estate...
228,Lai Tsui Court,Sham Shui Po,Kowloon West,22.334057,114.15447,Public Rental Housing,2019,Non-standard,2,Lai Pak House Lai Yeung House,1 300 As at 31.3.2021,14.25 - 35.84,Not Yet Formed,Kowloon West and Sai Kung District Tenancy Man...,Nice Property Management Ltd. 1/F Lai Tsui S...,Yue Xiu APT Parking Limited 27/F Seabright Pl...,,<a href= /en/common/pdf/global-elements/estate...
229,Fai Ming Estate,North,New Territories East,22.4811,114.1361,Public Rental Housing,2020,Non-Standard Block,2,Tai Fai House Sing Fai House,1 000 As at 31.3.2021,14.05 - 37.44,Not Yet Formed,Tai Po North & Shatin District Tenancy Manage...,Kai Fu Property Services Company Limited G/F ...,Kai Fu Property Services Company Limited G/F ...,-,<a href= /en/common/pdf/global-elements/estate...
230,Hoi Tat Estate,Sham Shui Po,Kowloon West,22.33005,114.14964,Public Rental Housing,2020 (Phase 1b),Non-standard,1,Hoi Wing House,2 300 As at 31.3.2021,14.05 - 35.88,Not Yet Formed,District Tenancy Management Office of Kowloon ...,China Overseas Property Services Limited G/F ...,China Overseas Property Services Limited G/F ...,,<a href= /en/common/pdf/global-elements/estate...


In [5]:
print("Shape: ", result2.shape)

Shape:  (102, 3)


### Explore and cluster the neighborhoods in Hong Kong
Get the latitude and longitude values of Hong Kong.

In [17]:
address = "Hong Kong"

geolocator = Nominatim(user_agent="HK_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of HK city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of HK city are 22.2793278, 114.1628131.


### Create a map of the whole HK with neighborhoods superimposed on top.

In [18]:
# create map of HK using latitude and longitude values
map_HK = folium.Map(location=[latitude, longitude], zoom_start=10)
map_HK

In [19]:
for lat, lng, borough, neighborhood in zip(
        df['Map_Latitude'], 
        df['Map_Longitude'], 
        df['District_Name'], 
        df['Estate_Name']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_HK)  

map_HK

### Map of a part of Hong Kong

In [48]:
df_TuenMun = df[df['District_Name'].str.contains("Tuen Mun")].reset_index(drop=True)
df_TuenMun.head()

,Estate_Name,District_Name,Region_Name,Map_Latitude,Map_Longitude,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,Property_Management,Carpark_Management,Estate_Website,Further_Information
0,Wu King Estate,Tuen Mun,New Territories West,22.37652,113.96507,Public Rental Housing,1982,Twin Tower,6,Wu Boon House Wu Fai House Wu Kwong House Wu P...,4 400 As at 31.3.2021,33.2 - 40.0,Formed,Wu King Estate Office G/F Wu Yuet House Wu K...,Wu King Estate Office G/F Wu Yuet House Wu K...,Yue Xiu APT Parking Ltd 28/F Seabright Plaza ...,-,<a href= /en/common/pdf/global-elements/estate...
1,Po Tin Estate,Tuen Mun,New Territories West,22.41024,113.96667,Public Rental Housing / Interim Housing,2000,Interim Housing,9,Block 1 Block 2 Block 3 Block 4 Block 5 Bloc...,4 700 As at 31.3.2021,8.2 - 28.3,Not Yet Formed,Tuen Mun North District Tenancy Management Off...,Easy Living Consultant Ltd. G/F Block 6 Po T...,Owned and operated by private corporation,-,* Includes flats/ households / authorized popu...
2,Sam Shing Estate,Tuen Mun,New Territories West,22.38107,113.97811,Public Rental Housing,1980,Double H Old Slab,3,Chun Yu House Fung Yu House Moon Yu House,1 800 As at 31.3.2021,28.6 - 54.8,Formed,Sam Shing Estate Office 1/F Fung Yu House Sa...,Sam Shing Estate Office 1/F Fung Yu House Sa...,Owned and operated by private corporation,-,<a href= /en/common/pdf/global-elements/estate...
3,Yau Oi Estate,Tuen Mun,New Territories West,22.38828,113.97354,Public Rental Housing,1980,Double H Old Slab Triple H Twin Tower,11,Oi Chi House Oi Lai House Oi Lim House Oi Lok ...,9 200 As at 31.3.2021,23.2 - 67.9,Formed,Yau Oi Estate Office Unit 101-103 & 105-108 G...,Yau Oi Estate Office Unit 101-103 & 105-108 G...,Owned and operated by private corporation,-,-
4,Tin King Estate,Tuen Mun,New Territories West,22.40587,113.96399,Tenants Purchase Scheme,1989,New Slab Trident 3 Trident 4,4,Tin Lok House Tin Tsui House Tin Tun House Tin...,600 * As at 31.3.2021,Gross Floor Areas (sqm):40.8 - 68.2; Saleable ...,Not applicable (Owners' Corporation Formed),Tuen Mun East District Tenancy Management Offi...,Appointed by Owners' Corporation,Owned and operated by private corporation,-,* Includes flats/households/authorized populat...


### Define Foursquare Credentials and Version
On the public repository on Github, I remove this field for the privacy!

In [77]:
CLIENT_ID = 'AEK3MFA03ZBAJW24E5LPAZYLGIMMQAAHG1TURPOXDSNP5E50'
CLIENT_SECRET = 'IMUP0XGDXALKLINWSMSYSSAJ4BOLNN5UV2LWKY2IEG5YJSVK'
VERSION = '20210717'


### Explore the first neighborhood in our data frame "df_TuenMun"

In [78]:
neighborhood_name = df.loc[0, 'Estate_Name']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'Tak Long Estate'.


In [79]:
neighborhood_latitude = df.loc[0, 'Map_Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Map_Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Tak Long Estate are 22.330105, 114.2031.


Get the top 100 venues that are in Tuen Mun Estates within a radius of 500 meters.

In [80]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results3 = requests.get(url).json()
# results3

In [81]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [82]:
venues = results3['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-82-190189955f46>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,McDonald’s (麥當勞),Fast Food Restaurant,22.331140,114.202559
1,Pacific Coffee (太平洋咖啡),Coffee Shop,22.326558,114.205666
2,McDonald's (麥當勞),Fast Food Restaurant,22.329503,114.205932
3,Golden Phoenix Restaurant (金鳳大餐廳),Steakhouse,22.331438,114.202805
4,潮家海鮮酒家,Cantonese Restaurant,22.331591,114.202853
5,Beacon Café (寶峰餐廳),Café,22.331921,114.202940
6,Tokachi Gyuu Japanese Restaurant (十勝牛和食料理),Japanese Restaurant,22.330795,114.204966
7,Ching Long Shopping Centre (晴朗商場),Shopping Mall,22.331981,114.202711
8,PARKnSHOP (百佳),Supermarket,22.329606,114.205939
9,MTR Kai Tak Station (港鐵啟德站),Metro Station,22.330946,114.199210


### Explore neighborhoods in a part of Tuen Mun
We are working on the data frame df_TuenMun

In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Estate_Name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues

In [100]:
TuenMun_venues = getNearbyVenues(names=df['Estate_Name'],
                                   latitudes=df['Map_Latitude'],
                                   longitudes=df['Map_Longitude']
                                  )

In [101]:
TuenMun_venues.head()

,Estate_Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tak Long Estate,22.330105,114.2031,McDonald’s (麥當勞),22.331140,114.202559,Fast Food Restaurant
1,Tak Long Estate,22.330105,114.2031,Pacific Coffee (太平洋咖啡),22.326558,114.205666,Coffee Shop
2,Tak Long Estate,22.330105,114.2031,McDonald's (麥當勞),22.329503,114.205932,Fast Food Restaurant
3,Tak Long Estate,22.330105,114.2031,Golden Phoenix Restaurant (金鳳大餐廳),22.331438,114.202805,Steakhouse
4,Tak Long Estate,22.330105,114.2031,潮家海鮮酒家,22.331591,114.202853,Cantonese Restaurant


Let's check how many venues were returned for each neighborhood.

In [103]:
TuenMun_venues.groupby('Estate_Name').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Estate_Name,,,,,,
Ap Lei Chau Estate,29,29,29,29,29,29
Butterfly Estate,15,15,15,15,15,15
Chai Wan Estate,9,9,9,9,9,9
Chak On Estate,4,4,4,4,4,4
Cheung Ching Estate,20,20,20,20,20,20
...,...,...,...,...,...,...
Ying Tung Estate,10,10,10,10,10,10
Yiu On Estate,6,6,6,6,6,6
Yiu Tung Estate,25,25,25,25,25,25


In [104]:
TuenMun_venues.head()

,Estate_Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tak Long Estate,22.330105,114.2031,McDonald’s (麥當勞),22.331140,114.202559,Fast Food Restaurant
1,Tak Long Estate,22.330105,114.2031,Pacific Coffee (太平洋咖啡),22.326558,114.205666,Coffee Shop
2,Tak Long Estate,22.330105,114.2031,McDonald's (麥當勞),22.329503,114.205932,Fast Food Restaurant
3,Tak Long Estate,22.330105,114.2031,Golden Phoenix Restaurant (金鳳大餐廳),22.331438,114.202805,Steakhouse
4,Tak Long Estate,22.330105,114.2031,潮家海鮮酒家,22.331591,114.202853,Cantonese Restaurant


In [105]:
TuenMun_venues.head()

,Estate_Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tak Long Estate,22.330105,114.2031,McDonald’s (麥當勞),22.331140,114.202559,Fast Food Restaurant
1,Tak Long Estate,22.330105,114.2031,Pacific Coffee (太平洋咖啡),22.326558,114.205666,Coffee Shop
2,Tak Long Estate,22.330105,114.2031,McDonald's (麥當勞),22.329503,114.205932,Fast Food Restaurant
3,Tak Long Estate,22.330105,114.2031,Golden Phoenix Restaurant (金鳳大餐廳),22.331438,114.202805,Steakhouse
4,Tak Long Estate,22.330105,114.2031,潮家海鮮酒家,22.331591,114.202853,Cantonese Restaurant


Let's find out how many unique categories can be curated from all the returned venues

In [106]:
print('There are {} uniques categories.'.format(len(TuenMun_venues['Venue Category'].unique())))

There are 211 uniques categories.


## Methodology <a name="methodology"></a>

In this project we will direct our effors on detecting areas of Tuen Mun district that have low restaruant density, particuarly those with low number of Chinese restaruants.

In first step we have collected the required data from HK open data - housing estates to gather where the end consumer are located / lived.

We also take advantages of Folium, which is a powerful library that provides nearby venues such as Yoga Studio, Afghan Restaurant, Airport, Airport Food Court, Airport Gate, Airport Lounge, Airport Service, Airport Terminal, American Restaurant, Antique Shop, Thrift / Vintage Store, Toy / Game Store, 'Trail, Train Station, Vegetarian / Vegan Restaurant, Video Game Store, Vietnamese Restaurant, Wine Bar, Wings Joint, and Women's Store.

In the final step we will focus on most promising areas and within those create **clusters of locations.**
K-means clustering is applied to the dataset for the exploration of different venues forming 3 clusters in which we have presented the top 10 venue categories for each cluster.

## Analysis <a name="analysis"></a>

In [108]:
# one hot encoding
TuenMun_onehot = pd.get_dummies(TuenMun_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TuenMun_onehot['Estate_Name'] = TuenMun_venues['Estate_Name'] 

# move neighborhood column to the first column
fixed_columns = [TuenMun_onehot.columns[-1]] + list(TuenMun_onehot.columns[:-1])
TuenMun_onehot = TuenMun_onehot[fixed_columns]

TuenMun_onehot.head()

,Estate_Name,ATM,Accessories Store,American Restaurant,Amphitheater,Arcade,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,...,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo
0,Tak Long Estate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Tak Long Estate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Tak Long Estate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Tak Long Estate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Tak Long Estate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
TuenMun_grouped = TuenMun_onehot.groupby('Estate_Name').mean().reset_index()
TuenMun_grouped.head()

,Estate_Name,ATM,Accessories Store,American Restaurant,Amphitheater,Arcade,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,...,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo
0,Ap Lei Chau Estate,0.0,0.034483,0.034483,0.00,0.0,0.0,0.034483,0.0,0.000000,...,0.0,0.00,0.000000,0.034483,0.0,0.0,0.0,0.0,0.0,0.0
1,Butterfly Estate,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Chai Wan Estate,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.111111,...,0.0,0.00,0.111111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Chak On Estate,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Cheung Ching Estate,0.0,0.000000,0.000000,0.05,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


Check the 10 most common venues in each neighborhood.

In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Estate_Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Estate_Name'] = TuenMun_grouped['Estate_Name']

for ind in np.arange(toronto_denc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TuenMun_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Estate_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ap Lei Chau Estate,Chinese Restaurant,Coffee Shop,Supermarket,Bus Station,Paper / Office Supplies Store,Seafood Restaurant,Hotel,Hotpot Restaurant,Shopping Mall,Sushi Restaurant
1,Butterfly Estate,Light Rail Station,Hotpot Restaurant,Donburi Restaurant,Fast Food Restaurant,Dessert Shop,Pub,Shopping Mall,Chinese Restaurant,Cha Chaan Teng,Garden
2,Chai Wan Estate,Athletics & Sports,Park,Seafood Restaurant,Bus Station,Chinese Restaurant,Cantonese Restaurant,Pool,Vietnamese Restaurant,Cha Chaan Teng,Pharmacy
3,Chak On Estate,Coffee Shop,Bus Station,Scenic Lookout,Vegetarian / Vegan Restaurant,Platform,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Pet Store
4,Cheung Ching Estate,Chinese Restaurant,Fast Food Restaurant,Convenience Store,Cha Chaan Teng,Paper / Office Supplies Store,Hong Kong Restaurant,Dessert Shop,Shopping Mall,Park,Supermarket


### Cluster neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [122]:
# set number of clusters
kclusters = 5

TuenMun_grouped_clustering = TuenMun_grouped.drop('Estate_Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TuenMun_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
kmeans

KMeans(n_clusters=5, random_state=0)

In [126]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels5', kmeans.labels_)

TuenMun_merged = df_TuenMun

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TuenMun_merged = TuenMun_merged.join(neighborhoods_venues_sorted.set_index('Estate_Name'), on='Estate_Name')

neighborhoods_venues_sorted


,Cluster Labels5,Cluster Labels4,Estate_Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,0,Ap Lei Chau Estate,Chinese Restaurant,Coffee Shop,Supermarket,Bus Station,Paper / Office Supplies Store,Seafood Restaurant,Hotel,Hotpot Restaurant,Shopping Mall,Sushi Restaurant
1,0,0,Butterfly Estate,Light Rail Station,Hotpot Restaurant,Donburi Restaurant,Fast Food Restaurant,Dessert Shop,Pub,Shopping Mall,Chinese Restaurant,Cha Chaan Teng,Garden
2,3,3,Chai Wan Estate,Athletics & Sports,Park,Seafood Restaurant,Bus Station,Chinese Restaurant,Cantonese Restaurant,Pool,Vietnamese Restaurant,Cha Chaan Teng,Pharmacy
3,3,3,Chak On Estate,Coffee Shop,Bus Station,Scenic Lookout,Vegetarian / Vegan Restaurant,Platform,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Pet Store
4,1,1,Cheung Ching Estate,Chinese Restaurant,Fast Food Restaurant,Convenience Store,Cha Chaan Teng,Paper / Office Supplies Store,Hong Kong Restaurant,Dessert Shop,Shopping Mall,Park,Supermarket
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0,0,Ying Tung Estate,Supermarket,Bus Station,Butcher,Café,Cantonese Restaurant,Hakka Restaurant,Coffee Shop,Bus Stop,Recreation Center,Fast Food Restaurant
226,3,3,Yiu On Estate,Bus Station,Track Stadium,Market,Playground,Chinese Restaurant,ATM,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant
227,0,0,Yiu Tung Estate,Fast Food Restaurant,Snack Place,Tram Station,Cha Chaan Teng,Chinese Restaurant,Farmers Market,Pizza Place,Hong Kong Restaurant,Convenience Store,Toy / Game Store
228,3,3,Yue Wan Estate,Park,Steakhouse,Seafood Restaurant,Athletics & Sports,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Food Court,Pizza Place,Pastry Shop


In [127]:
TuenMun_merged.head() # check the last columns!

,Estate_Name,District_Name,Region_Name,Map_Latitude,Map_Longitude,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wu King Estate,Tuen Mun,New Territories West,22.37652,113.96507,Public Rental Housing,1982,Twin Tower,6,Wu Boon House Wu Fai House Wu Kwong House Wu P...,...,Light Rail Station,Fast Food Restaurant,Shopping Mall,Cha Chaan Teng,Sushi Restaurant,Hotpot Restaurant,Dumpling Restaurant,Donburi Restaurant,Pub,Dessert Shop
1,Po Tin Estate,Tuen Mun,New Territories West,22.41024,113.96667,Public Rental Housing / Interim Housing,2000,Interim Housing,9,Block 1 Block 2 Block 3 Block 4 Block 5 Bloc...,...,Shopping Mall,Cha Chaan Teng,Supermarket,Sushi Restaurant,Cantonese Restaurant,Taiwanese Restaurant,Pizza Place,Convenience Store,Sports Club,Shopping Plaza
2,Sam Shing Estate,Tuen Mun,New Territories West,22.38107,113.97811,Public Rental Housing,1980,Double H Old Slab,3,Chun Yu House Fung Yu House Moon Yu House,...,Seafood Restaurant,Fast Food Restaurant,Light Rail Station,Bus Station,Market,Chinese Restaurant,Playground,Shopping Mall,Soccer Field,Pizza Place
3,Yau Oi Estate,Tuen Mun,New Territories West,22.38828,113.97354,Public Rental Housing,1980,Double H Old Slab Triple H Twin Tower,11,Oi Chi House Oi Lai House Oi Lim House Oi Lok ...,...,Zoo,Park,Bus Stop,Ramen Restaurant,Athletics & Sports,Fast Food Restaurant,Cantonese Restaurant,Pizza Place,Pastry Shop,Peking Duck Restaurant
4,Tin King Estate,Tuen Mun,New Territories West,22.40587,113.96399,Tenants Purchase Scheme,1989,New Slab Trident 3 Trident 4,4,Tin Lok House Tin Tsui House Tin Tun House Tin...,...,Light Rail Station,Cha Chaan Teng,Hong Kong Restaurant,Fast Food Restaurant,Sushi Restaurant,Market,Taiwanese Restaurant,Playground,Shopping Mall,Sports Club


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

let's visualize the resulting clusters

In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        TuenMun_merged['Map_Latitude'], 
        TuenMun_merged['Map_Longitude'], 
        TuenMun_merged['Estate_Name'], 
        TuenMun_merged['Cluster Labels4']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 1

In [132]:
TuenMun_merged.loc[TuenMun_merged['Cluster Labels4'] == 0, TuenMun_merged.columns[[1] + list(range(5, TuenMun_merged.shape[1]))]]

,District_Name,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tuen Mun,Public Rental Housing,1982,Twin Tower,6,Wu Boon House Wu Fai House Wu Kwong House Wu P...,4 400 As at 31.3.2021,33.2 - 40.0,Formed,Wu King Estate Office G/F Wu Yuet House Wu K...,...,Light Rail Station,Fast Food Restaurant,Shopping Mall,Cha Chaan Teng,Sushi Restaurant,Hotpot Restaurant,Dumpling Restaurant,Donburi Restaurant,Pub,Dessert Shop
1,Tuen Mun,Public Rental Housing / Interim Housing,2000,Interim Housing,9,Block 1 Block 2 Block 3 Block 4 Block 5 Bloc...,4 700 As at 31.3.2021,8.2 - 28.3,Not Yet Formed,Tuen Mun North District Tenancy Management Off...,...,Shopping Mall,Cha Chaan Teng,Supermarket,Sushi Restaurant,Cantonese Restaurant,Taiwanese Restaurant,Pizza Place,Convenience Store,Sports Club,Shopping Plaza
2,Tuen Mun,Public Rental Housing,1980,Double H Old Slab,3,Chun Yu House Fung Yu House Moon Yu House,1 800 As at 31.3.2021,28.6 - 54.8,Formed,Sam Shing Estate Office 1/F Fung Yu House Sa...,...,Seafood Restaurant,Fast Food Restaurant,Light Rail Station,Bus Station,Market,Chinese Restaurant,Playground,Shopping Mall,Soccer Field,Pizza Place
3,Tuen Mun,Public Rental Housing,1980,Double H Old Slab Triple H Twin Tower,11,Oi Chi House Oi Lai House Oi Lim House Oi Lok ...,9 200 As at 31.3.2021,23.2 - 67.9,Formed,Yau Oi Estate Office Unit 101-103 & 105-108 G...,...,Zoo,Park,Bus Stop,Ramen Restaurant,Athletics & Sports,Fast Food Restaurant,Cantonese Restaurant,Pizza Place,Pastry Shop,Peking Duck Restaurant
4,Tuen Mun,Tenants Purchase Scheme,1989,New Slab Trident 3 Trident 4,4,Tin Lok House Tin Tsui House Tin Tun House Tin...,600 * As at 31.3.2021,Gross Floor Areas (sqm):40.8 - 68.2; Saleable ...,Not applicable (Owners' Corporation Formed),Tuen Mun East District Tenancy Management Offi...,...,Light Rail Station,Cha Chaan Teng,Hong Kong Restaurant,Fast Food Restaurant,Sushi Restaurant,Market,Taiwanese Restaurant,Playground,Shopping Mall,Sports Club
6,Tuen Mun,Tenants Purchase Scheme,1989,Trident 4,4,Hong Sang House Lok Sang House Tai Sang House ...,400 * As at 31.3.2021,Gross Floor Areas (sqm):25.0 - 71.9;Saleable A...,Not applicable (Owners' Corporation Formed),Tuen Mun East District Tenancy Management Offi...,...,Light Rail Station,Shopping Mall,Supermarket,Sports Club,Taiwanese Restaurant,Cantonese Restaurant,Soccer Stadium,Pizza Place,Convenience Store,Shopping Plaza
7,Tuen Mun,Tenants Purchase Scheme,1988,New Slab Trident 3 Trident 4,8,Leung Wah House Leung Chi House Leung Ying Hou...,2 000 * As at 31.3.2021,Gross Floor Areas (sqm):18.5 - 87.1; Saleable ...,Not applicable (Owners' Corporation Formed),Tuen Mun West District Tenancy Management Offi...,...,Light Rail Station,Shopping Mall,Cha Chaan Teng,Hong Kong Restaurant,Fast Food Restaurant,Market,Taiwanese Restaurant,Sushi Restaurant,Sports Club,Italian Restaurant
8,Tuen Mun,Public Rental Housing,1977 - 1980,Cruciform Old Slab,7,Hing Cheung House Hing Fai House Hing Ping Hou...,8 600 As at 31.3.2021,22.0 - 47.3,Formed,Tai Hing Estate Office No. 26-38 G/F Hing W...,...,Supermarket,Pizza Place,Light Rail Station,Convenience Store,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Pet Store
9,Tuen Mun,Tenants Purchase Scheme,1983,Double H Old Slab Trident 1 Trident 2,9,King Fu House King Kwai House King Wing House ...,4 600 * As at 31.3.2021,23.3 - 64.4,Not applicable (Owners' Corporation Formed),Tuen Mun West District Tenancy Management Offi...,...,Light Rail Station,Macanese Restaurant,Mountain,ATM,Pizza Place,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Pet Store
10,Tuen Mun,Publ

Cluster 2

In [136]:
TuenMun_merged.loc[TuenMun_merged['Cluster Labels4'] == 1, TuenMun_merged.columns[[1] + list(range(5, TuenMun_merged.shape[1]))]]

,District_Name,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Cluster 3

In [137]:
TuenMun_merged.loc[TuenMun_merged['Cluster Labels4'] == 2, TuenMun_merged.columns[[1] + list(range(5, TuenMun_merged.shape[1]))]]

,District_Name,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Tuen Mun,Public Rental Housing,2000,Concord 1 Harmony 1 Single Aspect Building,12,Kwan Tai House Mei Tai House Oi Tai House Sau ...,5 100 As at 31.3.2021,16.3 - 53.1,Formed,Tuen Mun North District Tenancy Management Off...,...,Burger Joint,Grocery Store,Shopping Mall,Fast Food Restaurant,Pier,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant


Cluster 4

In [138]:
TuenMun_merged.loc[TuenMun_merged['Cluster Labels4'] == 3, TuenMun_merged.columns[[1] + list(range(5, TuenMun_merged.shape[1]))]]

,District_Name,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Tuen Mun,Public Rental Housing,2013,Non-Standard,2,Kin Lung House Hong Lung House,1 000 As at 31.3.2021,14.05-37.08,Not Yet Formed,Tuen Mun North District Tenancy Management Off...,...,River,Athletics & Sports,Restaurant,Supermarket,Park,Pool,Bus Station,Light Rail Station,Ramen Restaurant,Record Shop


Cluster 5

In [139]:
TuenMun_merged.loc[TuenMun_merged['Cluster Labels4'] == 4, TuenMun_merged.columns[[1] + list(range(5, TuenMun_merged.shape[1]))]]

,District_Name,Type_of_Estate,Year_of_Intake,Type_of_Block,No_of_Blocks,Name_of_Block,No_of_Rental_Flats,Flat_Size_m2,Estate_Management_Advisory_Committee_EMAC,District_Tenancy_Management_Office_Estate_Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## Results and Discussion <a name="results"></a>

Analysis of each estate is carried out with the help of one-hot encoding which explore the top 10 common venues. 

After one hot encoding and venue exploration, we applied the k-means algorithm for k=5. It has identified 3 cluster centroids with five different colors. Cluster 4 and 5 are having only one estate while cluster 1 is having the maximum number of estates.


## Conclusion <a name="conclusion"></a>

The purpose of this project was to identify Tuen Mun district close to the center with a low number of restaurants to aid business stakeholders in narrowing down the search for the optimal location for a new restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis, and then generated an extensive collection of locations that satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed to create major zones of interest (containing the greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by b stakeholders.



Purpose of this project was to identify Berlin areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis (Kreuzberg and Friedrichshain), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.